# Revised case normalization for KSSG 2021 

Notes: 

- All cases (25 of 25) could be matched between the revised cases from DtoD and the BFS DB data of KSSG 2021.

In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/jovyan/work')

import os

from py.global_configs import *
from py.normalize import normalize

from src.service import bfs_cases_db_service as bfs_db

from src.utils.dataframe_utils import remove_duplicated_chops

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:

file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/KSSG_2021.xlsx'),
         'KSSG HQ',
         '2021',
         ['Änderungen_KSSG_2021'])

print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/KSSG_2021.xlsx', hospital_name_db='KSSG HQ', year='2021', sheets=['Änderungen_KSSG_2021'])


In [3]:
cols_to_rename = dict(COLUMNS_TO_RENAME)
# Replace 'admno' with 'fall nummer'
cols_to_rename.pop('admno')
cols_to_rename['fall nummer'] = CASE_ID_COL

df_revised_case_d2d = normalize(file_info, 0)

2022-10-18 12:05:28.420 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 1: discarded ICDs after validation {''}
2022-10-18 12:05:28.422 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 2: discarded ICDs after validation {''}
2022-10-18 12:05:28.423 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 3: discarded ICDs after validation {''}
2022-10-18 12:05:28.424 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 5: discarded ICDs after validation {''}


Read 25 cases for KSSG HQ 2021
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
old_pd              string
new_pd              string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


2022-10-18 12:05:28.425 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 7: discarded ICDs after validation {''}
2022-10-18 12:05:28.425 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 9: discarded ICDs after validation {''}
2022-10-18 12:05:28.426 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 10: discarded ICDs after validation {''}
2022-10-18 12:05:28.427 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 11: discarded ICDs after validation {''}
2022-10-18 12:05:28.428 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 12: discarded ICDs after validation {''}
2022-10-18 12:05:28.428 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 13: discarded ICDs after validation {''}
2022-10-18 12:05:28.429 | DEBUG    | src.utils.dataframe_utils:_validate_icd_codes:56 - row 17: discarded ICDs after validation {''}
2022-10-18 12:05:28.430 | DEBUG    | src.utils.dataframe_utils:_validat

In [4]:
df_revised_case_d2d

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,1112425128,F1B1A46BC1FEF112,M,56,5,3,H41C,K8031,K8031,M100,"[K8510, D6959]",[],[],[]
1,1112427844,59106C27B3536BE5,M,64,9,3,T62C,R5088,R5088,M100,"[I691, G819]",[],[],[]
2,1112441515,BB0033F7EF9CCB1F,W,70,3,3,R61D,C833,C833,M100,[D6110],[],[],[]
3,1112445586,73EFDADADF92D6A4,W,93,7,2,F62D,I5001,I5001,M100,[F050],[],[],[]
4,1112426216,FD14EA356F1A32E4,M,78,4,2,E77E,J128,J128,M100,[J4412],[J4482],[],[]
5,1112427071,C47C1FF3CA495488,W,70,2,2,I75C,M6286,M6286,M100,[F132],[],[],[]
6,1112430217,1E6CD6DFCCD600D9,M,87,9,3,E77D,J128,J128,M100,[E440],[E46],[890A32],[]
7,1112432613,C997F68564CDE89B,M,51,10,2,E77E,J128,J128,M100,[D803],[],[],[]
8,1112438704,5753DCD32E7CF5CA,W,71,5,3,B70D,I634,I634,M100,[G836],[G510],[],[]
9,1112448747,3A15CC21343AFF78,M,64,2,3,G48C,K912,K912,M100,[L8914],[],[],[]


# Match revised DtoD data with the bfs data from the database


In [5]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['1112425128', '1112427844', '1112441515', '1112445586',
       '1112426216', '1112427071', '1112430217', '1112432613',
       '1112438704', '1112448747', '1112450067', '1112443326',
       '1112448386', '1112454721', '1112482020', '1112465409',
       '1112458975', '1112459353', '1112471385', '1112497209',
       '1112510073', '1112518499', '1112503831', '1112486634',
       '1112488070'], dtype=object)

In [6]:
revised_case_id_stripped = [case_id.lstrip('0') for case_id in revised_case_id]

In [7]:
# change the case_id the same format as bfs data e.g. '0041194601' 
# revised_case_id_pad0 = [case_id.zfill(10) for case_id in revised_case_id]
# revised_case_id_pad0[0]
# df_revised_case_d2d['case_id'] = revised_case_id_pad0

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids_no_pad0(revised_case_id_stripped)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,1.152,456439,12,1112425128,F1B1A46BC1FEF112,56,0,M,5,3,0,0,0,2020-12-31,01,2021-01-05,00,H41C,H41,3
0,0.571,456987,12,1112427844,59106C27B3536BE5,64,0,M,9,3,0,0,0,2021-01-07,01,2021-01-16,06,T62C,T62,3
0,0.820,459226,12,1112441515,BB0033F7EF9CCB1F,70,0,W,3,3,0,0,0,2021-02-05,01,2021-02-08,00,R61D,R61,3
0,0.732,459866,12,1112445586,73EFDADADF92D6A4,93,0,W,7,3,0,0,0,2021-02-15,01,2021-02-22,00,F62D,F62,2
0,0.628,456681,12,1112426216,FD14EA356F1A32E4,78,0,M,4,3,0,0,0,2021-01-04,01,2021-01-08,00,E77E,E77,2


In [9]:
# create a new column for case_id_stripped with '0' for joining to the revised cases
revised_case_db['case_id_stripped']=  revised_case_db['case_id'].apply(lambda x: x.lstrip('0')) 
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,...,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl,case_id_stripped
0,1.152,456439,12,1112425128,F1B1A46BC1FEF112,56,0,M,5,3,...,0,0,2020-12-31,01,2021-01-05,00,H41C,H41,3,1112425128
0,0.571,456987,12,1112427844,59106C27B3536BE5,64,0,M,9,3,...,0,0,2021-01-07,01,2021-01-16,06,T62C,T62,3,1112427844
0,0.820,459226,12,1112441515,BB0033F7EF9CCB1F,70,0,W,3,3,...,0,0,2021-02-05,01,2021-02-08,00,R61D,R61,3,1112441515
0,0.732,459866,12,1112445586,73EFDADADF92D6A4,93,0,W,7,3,...,0,0,2021-02-15,01,2021-02-22,00,F62D,F62,2,1112445586
0,0.628,456681,12,1112426216,FD14EA356F1A32E4,78,0,M,4,3,...,0,0,2021-01-04,01,2021-01-08,00,E77E,E77,2,1112426216


In [10]:
df_revised_case_d2d['case_id_stripped']=  df_revised_case_d2d['case_id'].apply(lambda x: x.lstrip('0')) 
df_revised_case_d2d.head()


,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped
0,1112425128,F1B1A46BC1FEF112,M,56,5,3,H41C,K8031,K8031,M100,"[K8510, D6959]",[],[],[],1112425128
1,1112427844,59106C27B3536BE5,M,64,9,3,T62C,R5088,R5088,M100,"[I691, G819]",[],[],[],1112427844
2,1112441515,BB0033F7EF9CCB1F,W,70,3,3,R61D,C833,C833,M100,[D6110],[],[],[],1112441515
3,1112445586,73EFDADADF92D6A4,W,93,7,2,F62D,I5001,I5001,M100,[F050],[],[],[],1112445586
4,1112426216,FD14EA356F1A32E4,M,78,4,2,E77E,J128,J128,M100,[J4412],[J4482],[],[],1112426216


# Forming the normalized dataframe: Merge the selected revised case data with the matched from bfs data
  If there are multiple cases were found based on case_id, patient_id will be used to match once more  

In [11]:
# Check if there are multiple cases were found based on case_id, patient_id will be used to match once more
if revised_case_db.shape[0] <= len(revised_case_id):
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_id = revised_case_normalized.shape[0]
else:
    revised_case_d2d_subset = df_revised_case_d2d[COL_SUBSET_FROM_REVISED_CASES + ['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg']]
    revised_case_normalized = revised_case_d2d_subset.merge(revised_case_db, how ='inner', on=['case_id_stripped', 'patient_id', 'gender', 'age_years', 'duration_of_stay', 'pccl', 'drg'])
    num_match_case_patient_id = revised_case_normalized.shape[0]
print(revised_case_normalized.shape)
revised_case_normalized.head()

(25, 28)


,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,case_id_stripped,gender,age_years,...,age_days,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,adrg
0,K8031,K8031,M100,"[K8510, D6959]",[],[],[],1112425128,M,56,...,0,3,0,0,0,2020-12-31,01,2021-01-05,00,H41
1,R5088,R5088,M100,"[I691, G819]",[],[],[],1112427844,M,64,...,0,3,0,0,0,2021-01-07,01,2021-01-16,06,T62
2,C833,C833,M100,[D6110],[],[],[],1112441515,W,70,...,0,3,0,0,0,2021-02-05,01,2021-02-08,00,R61
3,I5001,I5001,M100,[F050],[],[],[],1112445586,W,93,...,0,3,0,0,0,2021-02-15,01,2021-02-22,00,F62
4,J128,J128,M100,[J4412],[J4482],[],[],1112426216,M,78,...,0,3,0,0,0,2021-01-04,01,2021-01-08,00,E77


In [12]:
df_revised_case_normalized = revised_case_normalized[COLS_TO_OUTPUT]
df_revised_case_normalized.head()

,aimedic_id,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,old_pd,new_pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,456439,1112425128,F1B1A46BC1FEF112,M,56,5,3,H41C,K8031,K8031,M100,"[K8510, D6959]",[],[],[]
1,456987,1112427844,59106C27B3536BE5,M,64,9,3,T62C,R5088,R5088,M100,"[I691, G819]",[],[],[]
2,459226,1112441515,BB0033F7EF9CCB1F,W,70,3,3,R61D,C833,C833,M100,[D6110],[],[],[]
3,459866,1112445586,73EFDADADF92D6A4,W,93,7,2,F62D,I5001,I5001,M100,[F050],[],[],[]
4,456681,1112426216,FD14EA356F1A32E4,M,78,4,2,E77E,J128,J128,M100,[J4412],[J4482],[],[]


# Export the normalized revised cases with the neccessary columns to a csv file


In [13]:

hospital = file_info.hospital_name_db.replace(' ', '_') # replace spaces with '_' for the file name
hospital_year = f'{hospital}_{file_info.year}'

output_folder = os.path.join(ROOT_DIR, 'normalized_revision_data/')

if not os.path.exists(output_folder):
    os.mkdir(output_folder)

output_path = os.path.join(ROOT_DIR, 'normalized_revision_data/') + hospital_year + '.csv'

df_revised_case_normalized.to_csv(output_path)


# Report

A small summary of:
- how many cases could be matched in the DB, and based on which columns
- how many cases were “correctly” revised, in terms of upcoding (still need to do after the `grouper` is available for python)

In [14]:
if revised_case_db.shape[0] <= len(revised_case_id):
    print(f'Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )
else:
    print(f'Based on the columns "case_id" and "Patient_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are {num_match_case_patient_id} out of {len(revised_case_id)} revised cases from DtoD that can be matched with the bfs database data for {file_info.hospital_name_db} {file_info.year}' )

Based on the columns "case_id", "gender, "age_years", "duration_of_stay", "pccl", "drg", there are 25 out of 25 revised cases from DtoD that can be matched with the bfs database data for KSSG HQ 2021


In [15]:
print('All the normalized revised cases are exported to here: {}'.format(output_path))

All the normalized revised cases are exported to here: /home/jovyan/work/src/revised_case_normalization/normalized_revision_data/KSSG_HQ_2021.csv
